In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# load dataframe previously stored
df = pd.read_pickle(path='./tweets_database.pickle')
print(df.info)

In [ ]:
# create a DF of nodes of the network
columns = ['user_id', 'username']
nodelist = pd.DataFrame(columns=columns)

# insert all tweet.users
nodelist['user_id'] = df['user_id_str']
nodelist['username'] = df['user_screen_name']
nodelist.set_index('user_id')

def get_users(user_id, username):
    """Get users from mentions, replies, retweets"""
    reduce = df.dropna(subset=[user_id, username])
    insert_list = []
    for index, row in reduce.iterrows():
        user = {'user_id': row[user_id], 'username': row[username]}
        insert_list.append(user)
    return pd.DataFrame(insert_list, columns=columns)

# insert all users of tweet.user_mentions
nodelist = nodelist.append(get_users('user_mention_id_str', 'user_mention_screen_name'), sort=False)

# insert all users of tweet.replies
nodelist = nodelist.append(get_users('in_reply_to_user_id_str', 'in_reply_to_screen_name'), sort=False)

# insert all users of tweet.retweets
nodelist = nodelist.append(get_users('retweeted_user_id_str', 'retweeted_user_screen_name'), sort=False)

# drop duplicates
nodelist.drop_duplicates(subset=columns, inplace=True)
print(nodelist.info)

# rename columns for Gephi
gephi_nodes = nodelist.rename(columns={'user_id': 'Id', 'username': 'Label'})
gephi_nodes.to_csv('nodelist.csv', index=False)

In [ ]:
# Create a DF of directed edges connecting the nodes of the network
columns = ['source_id', 'target_id']
edgelist = pd.DataFrame(columns=columns)

def get_interactions(source_id, target_id):
    """Get interactions from mentions, replies, retweets"""
    reduce = df.dropna(subset=[source_id, target_id])
    insert_list = []
    for index, row in reduce.iterrows():
        interaction = {'source_id': row[source_id], 'target_id': row[target_id]}
        insert_list.append(interaction)
    return pd.DataFrame(insert_list, columns=columns)

# insert all user mentions, source=original, target=user mentioning another user
edgelist = edgelist.append(get_interactions(source_id='user_mention_id_str', target_id='user_id_str'))

# insert all replies, source=original, target=user replying to another user
edgelist = edgelist.append(get_interactions(source_id='in_reply_to_user_id_str', target_id='user_id_str'))

# insert all retweets, source=original, target=user retweeting another user
edgelist = edgelist.append(get_interactions(source_id='retweeted_user_id_str', target_id='user_id_str'))

# drop duplicates
edgelist.drop_duplicates(subset=['source_id', 'target_id'], inplace=True)

#TODO: remove self-references

print(edgelist.info)

# rename columns for Gephi
gephi_edges = edgelist.rename(columns={'source_id': 'Source', 'target_id': 'Target'})
gephi_edges.to_csv('edgelist.csv', index=False)

In [ ]:
# TODO: drop users that do not have any retweets, replies, or mentions to keep the graph lighter